## Detail - Level 1

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms


# Load FashionMNIST dataset
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.FashionMNIST(root='data',  train=True, 
                                             download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.FashionMNIST(root='data', train=False, 
                                            download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# Define the MLP model
model = nn.Sequential(
    nn.Flatten(), 
    nn.Linear(784, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training the model
max_epoch = 5
for epoch in range(max_epoch):
    for i, (inputs, labels) in enumerate(trainloader, 0):

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch + 1}/{max_epoch}]")

Epoch [1/5]
Epoch [2/5]
Epoch [3/5]
Epoch [4/5]
Epoch [5/5]


In [3]:
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"accuracy: {accuracy}")

accuracy: 83.95


## Detail - Level 2

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Load FashionMNIST dataset
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.FashionMNIST(root='data', train=True, 
                                             download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.FashionMNIST(root='data', train=False, 
                                            download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# Define the MLP model
model = nn.Sequential(
    nn.Flatten(), 
    nn.Linear(784, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training the model
max_epoch = 5
for epoch in range(max_epoch):
    running_loss = 0.0
    correct = 0   # to track number of correct predictions
    total = 0     # to track total number of samples

    for i, (inputs, labels) in enumerate(trainloader, 0):

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)

        loss.backward()
        optimizer.step()

        # Determine class predictions and track accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # accumulate loss
        running_loss += loss.item()

    epoch_accuracy = 100 * correct / total
    running_loss = running_loss / (i + 1)
    
    print(f"Epoch [{epoch + 1}/{max_epoch}], Loss: {running_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

Epoch [1/5], Loss: 0.7918, Accuracy: 74.58%
Epoch [2/5], Loss: 0.5199, Accuracy: 81.78%
Epoch [3/5], Loss: 0.4712, Accuracy: 83.41%
Epoch [4/5], Loss: 0.4440, Accuracy: 84.39%
Epoch [5/5], Loss: 0.4243, Accuracy: 84.96%


## Detail - Level 3

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Load FashionMNIST dataset
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.FashionMNIST(root='data', train=True, 
                                             download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.FashionMNIST(root='data', train=False, 
                                            download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# Define the MLP model
model = nn.Sequential(
    nn.Flatten(), 
    nn.Linear(784, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Function to compute loss and accuracy for test set
def evaluate(model, testloader, loss_fn):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:

            outputs = model(images)
            loss = loss_fn(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return test_loss / len(testloader), accuracy

# Training the model
for epoch in range(5):
    running_loss = 0.0
    correct = 0   # to track number of correct predictions
    total = 0     # to track total number of samples

    for i, (inputs, labels) in enumerate(trainloader, 0):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        # Determine class predictions and track accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        # accumulate loss
        running_loss += loss.item()

    epoch_accuracy = 100 * correct / total
    running_loss = running_loss / (i + 1)    
    test_loss, test_accuracy = evaluate(model, 
                                        testloader, 
                                        loss_fn)
    
    print(f"Epoch [{epoch + 1}/{max_epoch}], Loss: {running_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")

print('Finished Training')

Epoch [1/5], Loss: 0.7873, Accuracy: 74.56%, Test Loss: 0.5856, Test Accuracy: 78.82%
Epoch [2/5], Loss: 0.5193, Accuracy: 81.73%, Test Loss: 0.5192, Test Accuracy: 81.01%
Epoch [3/5], Loss: 0.4705, Accuracy: 83.42%, Test Loss: 0.4893, Test Accuracy: 82.18%
Epoch [4/5], Loss: 0.4431, Accuracy: 84.41%, Test Loss: 0.4675, Test Accuracy: 82.96%
Epoch [5/5], Loss: 0.4240, Accuracy: 85.10%, Test Loss: 0.4614, Test Accuracy: 82.92%
Finished Training
